In [12]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp_data_analysis.tables import tbls
import calitp_data_analysis.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
from rt_analysis import rt_filter_map_plot
# import build_speedmaps_index

from IPython.display import display, Markdown, Latex, HTML
import json

In [13]:
## parameters cell
itp_id = 182

In [14]:
# analysis_date = build_speedmaps_index.ANALYSIS_DATE
analysis_date = dt.date(2023, 4, 12)

In [15]:
%%capture_parameters
human_date = analysis_date.strftime('%B %d %Y (%A)')
human_date

{"human_date": "April 12 2023 (Wednesday)"}


In [16]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [17]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

{"organization_name": "Los Angeles County Metropolitan Transportation Authority"}


# {organization_name}

## About These Maps:

* Each map shows bus (and rail, if applicable) speeds for {organization_name}, with a map each for the morning peak, midday, and afternoon peak periods on {human_date}.
* On the map, routes are split into segments corresponding to the distance between two stops, allowing you to focus on specific portions of the route experiencing a slowdown.
* Route segments are arrow-shaped to indicate direction of travel. Thicker segments indicate more frequent transit routes.
* Hover over, or click, a segment with your mouse to see the exact speed, route name, and transit service frequency in that segment. 
    * Higher-frequency routes (3+ trips per hour) are especially important, since those slowdowns correspond to more vehicles, and riders, delayed in traffic.

## Morning Peak

In [18]:
%%capture
rt_day.set_filter(start_time='06:00', end_time='09:00')

In [19]:
## avoid papermill errors when map fails to render
m = None

In [20]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds', no_title=True, shn=True)

In [22]:
#m